In [ ]:
import os
os.environ["CURL_CA_BUNDLE"]=""

In [1]:
import pandas as pd

In [2]:
# tigdata = pd.read_csv("tig.csv")

In [3]:
# tigdata.shape

In [4]:
# type(tigdata)

In [5]:
# train, test = train_test_split(tigdata, test_size=0.2)

In [6]:
# train.to_csv("t_tig_train.csv")

In [7]:
# test.to_csv("t_tig_test.csv")

In [8]:
from datasets import load_dataset

In [9]:
dataset = load_dataset('csv', data_files={'train': 'tig_train.csv', 'test': 'tig_test.csv'})

Using custom data configuration default-f84e6cb3d4bed99f
Reusing dataset csv (C:\Users\Estifanos Daniel\.cache\huggingface\datasets\csv\default-f84e6cb3d4bed99f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 39499
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 9875
    })
})

In [11]:
# newtrain = pd.DataFrame.from_dict(dataset['train'])
# newtest = pd.DataFrame.from_dict(dataset['test'])

In [12]:
# newtrain.drop('Unnamed: 0', axis=1, inplace=True)
# newtrain.to_csv("newtrain.csv")

In [13]:
# newtest.drop('Unnamed: 0', axis=1, inplace=True)
# newtest.to_csv("newtest.csv")

In [14]:
dataset["train"][309]

{'label': 1, 'text': 'ንምንታይ ዳኣልኩም ገጾም ሸፍንኩሞ ክክክ'}

In [15]:
tokenizer_folder = '../../../tokenizer/tokenized/tigrigna' # tokenizer folder
trained_folder =   '../../../trained_model/tigrigna/checkpoint-290000' # trained folder

In [16]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=512)

In [17]:
tokenizer('ሰላም')

{'input_ids': [0, 333, 3261, 2], 'attention_mask': [1, 1, 1, 1]}

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [19]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Estifanos Daniel\.cache\huggingface\datasets\csv\default-f84e6cb3d4bed99f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-b31e01d7387c8cde.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

In [20]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [21]:
train_dataset.unique('label')

[0, 1]

In [22]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(trained_folder, num_labels=2)

Some weights of the model checkpoint at ../../../trained_model/tigrigna/checkpoint-290000 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../../../trained_model/tigrigna/checkpoint-290000 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.b

In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="finetuned")

In [24]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=16,
    per_device_train_batch_size=4,
    save_steps=5000,
    output_dir="finetuned", 
    evaluation_strategy="epoch")

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
E:\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 39499
  Num Epochs = 16
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 158000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.521300,0.342830,0.865418
2,0.599000,0.501496,0.868557
3,0.500200,0.400768,0.876152
4,0.567800,0.757231,0.809114
5,0.495900,0.443296,0.896405
6,0.402900,0.378024,0.910076
7,0.391500,0.424122,0.903696
8,0.401200,0.460451,0.880506
9,0.377200,0.389632,0.912810
10,0.363500,0.385028,0.914633


Saving model checkpoint to finetuned\checkpoint-5000
Configuration saved in finetuned\checkpoint-5000\config.json
Model weights saved in finetuned\checkpoint-5000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9875
  Batch size = 8
Saving model checkpoint to finetuned\checkpoint-10000
Configuration saved in finetuned\checkpoint-10000\config.json
Model weights saved in finetuned\checkpoint-10000\pytorch_model.bin
Saving model checkpoint to finetuned\checkpoint-15000
Configuration saved in finetuned\checkpoint-15000\config.json
Model weights saved in finetuned\checkpoint-15000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.

  Num examples = 9875
  Batch size = 8
Saving model checkpoint to finetuned\checkpoint-120000
Configuration saved in finetuned\checkpoint-120000\config.json
Model weights saved in finetuned\checkpoint-120000\pytorch_model.bin
Saving model checkpoint to finetuned\checkpoint-125000
Configuration saved in finetuned\checkpoint-125000\config.json
Model weights saved in finetuned\checkpoint-125000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9875
  Batch size = 8
Saving model checkpoint to finetuned\checkpoint-130000
Configuration saved in finetuned\checkpoint-130000\config.json
Model weights saved in finetuned\checkpoint-130000\pytorch_model.bin
Saving model checkpoint to finetuned\checkpoint-135000
Configurati

TrainOutput(global_step=158000, training_loss=0.39059357322016847, metrics={'train_runtime': 35229.3989, 'train_samples_per_second': 17.939, 'train_steps_per_second': 4.485, 'total_flos': 8.37172764722135e+16, 'train_loss': 0.39059357322016847, 'epoch': 16.0})